In [2]:
!pip install -q langchain numexpr sentencepiece plotly


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [3]:
%reload_ext autoreload
%autoreload 2

In [4]:
from langchain_community.llms import HuggingFaceEndpoint
from langchain.chat_models import ChatHuggingFace
import pandas as pd
from dotenv import load_dotenv

load_dotenv(override=True)

from scripts.evaluation import load_benchmark
import datasets

In [5]:
from langchain.chat_models import ChatOpenAI

llm = HuggingFaceEndpoint(
    repo_id="NousResearch/Nous-Hermes-2-Mixtral-8x7B-DPO",
    max_new_tokens=3000,
)
llm = HuggingFaceEndpoint(
    repo_id="mistralai/Mixtral-8x7B-Instruct-v0.1",
    max_new_tokens=3000,
)

llm_engine_hf = ChatHuggingFace(llm=llm)
llm_engine_openai = ChatOpenAI(model="gpt-3.5-turbo-0125")

eval_chat_model = ChatOpenAI(model="gpt-4-1106-preview", temperature=0)
eval_model_name = "GPT4"


def call_llm_hf(input: str, stop=["Observation", "Final Answer"]) -> str:
    return llm_engine_hf.invoke(input, stop=stop).content


def call_llm_openai(input: str, stop=["Observation", "Final Answer"]) -> str:
    return llm_engine_openai.invoke(input, stop=stop).content


call_llm_openai("Please output 'Observation'")

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /Users/aymeric/.cache/huggingface/token
Login successful
Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /Users/aymeric/.cache/huggingface/token
Login successful


/Users/aymeric/venvs/ml/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


''

In [6]:
OPENAI_MODEL_ID = "gpt-4-0125-preview"

In [7]:
eval_df = load_benchmark()
eval_df["true_answer"] = eval_df["true_answer"].astype(str)
eval_ds = datasets.Dataset.from_pandas(eval_df)

OUTPUT_DIR = "output"

### Define tools

In [4]:
!pip install wikipedia

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11680 sha256=65a6dc62a49a098abc4f071ad7f3e98fa9cb7c14cad47d4c82abc57fab8a35ee
  Stored in directory: /Users/aymeric/Library/Caches/pip/wheels/8f/ab/cb/45ccc40522d3a1c41e1d2ad53b8f33a62f394011ec38cd71c6
Successfully built wikipedia

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [12]:
from transformers.tools.base import Tool

from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=100)
langchain_tool = WikipediaQueryRun(api_wrapper=api_wrapper)

tool = Tool.from_langchain(langchain_tool)

In [11]:
tool.output_type

str

In [8]:
from transformers.tools.default_tools import CalculatorTool, Tool

from langchain_community.utilities import SerpAPIWrapper

params = {
    "engine": "bing",
    "gl": "us",
    "hl": "en",
}
langchain_serpapi = SerpAPIWrapper(params=params)


class SearchTool(Tool):
    name = "search"
    description = "Search the web for information."

    inputs = {"query": {"description": "the query", "type": str}}
    output_type = str

    def __call__(self, query: str) -> str:
        return langchain_serpapi.run(query)

In [9]:
TOOLBOX = [CalculatorTool(), SearchTool()]

In [10]:
CalculatorTool()("2+2")

'4'

# Define agents

In [41]:
from transformers import CodeAgent, ReactAgent

code_agent = CodeAgent(llm_callable=call_llm_hf, tools=TOOLBOX)
react_agent_openai = ReactAgent(
    llm_callable=call_llm_openai, tools=TOOLBOX, max_iterations=7
)
react_agent_hf = ReactAgent(llm_callable=call_llm_hf, tools=TOOLBOX, max_iterations=7)

agents = {
    # "code": code_agent,
    # "react_text_openai": react_agent_openai,
    "react_text_mixtral3": react_agent_hf,
    # "vanilla_llm": llm,
}

In [42]:
async def call_transformers(agent, question: str) -> str:
    result = agent.run(question)
    return {
        "output": str(result),
        "intermediate_steps": agent.memory.copy(),
    }


async def call_vanilla(agent, question: str) -> str:
    result = agent(question)
    return {
        "output": str(result),
        "intermediate_steps": [],
    }

In [43]:
react_agent_hf.log

<Logger transformers.tools.agents (INFO)>

In [44]:
react_agent_hf.log.info("OK")

OK


In [45]:
result = await call_transformers(react_agent_hf, "I should multiply 2 by 3.6452.")
print(result)

=====Calling LLM with these messages:=====


Solve the following task as best you can. You have access to the following tools:


- calculator: This is a tool that calculates. It can be used to perform simple arithmetic operations.
    Takes inputs: {'expression': {'type': <class 'str'>, 'description': "The expression to be evaluated.The variables used CANNOT be placeholders like 'x' or 'mike's age', they must be numbers"}}

- search: Search the web for information.
    Takes inputs: {'query': {'description': 'the query', 'type': <class 'str'>}}

The way you use the tools is by specifying a json blob.
Specifically, this json should have a `action` key (name of the tool to use) and a `action_input` key (input to the tool).

The value in the "action" field should belong to this list: 'calculator', 'search'.

The $ACTION_JSON_BLOB should only contain a SINGLE action, do NOT return a list of multiple actions. It should be formatted in json. Do not try to escape special characters. Here is an example of a valid $ACTION_JSON_BLOB:
Actio

OIDOIOINCOZ
{'max_new_tokens': 3000, 'top_k': None, 'top_p': 0.95, 'typical_p': 0.95, 'temperature': 0.8, 'repetition_penalty': None, 'return_full_text': False, 'truncate': None, 'stop_sequences': ['Observation:'], 'seed': None, 'do_sample': False, 'watermark': False} None
OIDOIOINCOZ
{'max_new_tokens': 3000, 'top_k': None, 'top_p': 0.95, 'typical_p': 0.95, 'temperature': 0.8, 'repetition_penalty': None, 'return_full_text': False, 'truncate': None, 'stop_sequences': ['Observation:'], 'seed': None, 'do_sample': False, 'watermark': False} None
{'output': '7.2904', 'intermediate_steps': ['Solve the following task as best you can. You have access to the following tools:\n\n\n- calculator: This is a tool that calculates. It can be used to perform simple arithmetic operations.\n    Takes inputs: {\'expression\': {\'type\': <class \'str\'>, \'description\': "The expression to be evaluated.The variables used CANNOT be placeholders like \'x\' or \'mike\'s age\', they must be numbers"}}\n\n- sea

Cell below removes empty answers ("None") 👇👇

In [46]:
from scripts.run_agents import run_full_tests

await run_full_tests(
    eval_ds,
    agents,
    output_folder=OUTPUT_DIR,
    agent_call_function=call_transformers,
    key_for_answer="true_answer",
)

  0%|          | 0/90 [00:00<?, ?it/s]=====Calling LLM with these messages:=====
Solve the following task as best you can. You have access to the following tools:


- calculator: This is a tool that calculates. It can be used to perform simple arithmetic operations.
    Takes inputs: {'expression': {'type': <class 'str'>, 'description': "The expression to be evaluated.The variables used CANNOT be placeholders like 'x' or 'mike's age', they must be numbers"}}

- search: Search the web for information.
    Takes inputs: {'query': {'description': 'the query', 'type': <class 'str'>}}

The way you use the tools is by specifying a json blob.
Specifically, this json should have a `action` key (name of the tool to use) and a `action_input` key (input to the tool).

The value in the "action" field should belong to this list: 'calculator', 'search'.

The $ACTION_JSON_BLOB should only contain a SINGLE action, do NOT return a list of multiple actions. It should be formatted in json. Do not try to 

OIDOIOINCOZ
{'max_new_tokens': 3000, 'top_k': None, 'top_p': 0.95, 'typical_p': 0.95, 'temperature': 0.8, 'repetition_penalty': None, 'return_full_text': False, 'truncate': None, 'stop_sequences': ['Observation:'], 'seed': None, 'do_sample': False, 'watermark': False} None


=====Output message of the LLM:=====
 Thought: To solve this problem, I first need to calculate the number of shells Kyle found. I can then determine how many shells Leigh collected.

Action:
{
  "action": "calculator",
  "action_input": {
    "expression": "2 * 24"
  }
}




==Result==
Observation: 48
=====Calling LLM with these messages:=====
Solve the following task as best you can. You have access to the following tools:


- calculator: This is a tool that calculates. It can be used to perform simple arithmetic operations.
    Takes inputs: {'expression': {'type': <class 'str'>, 'description': "The expression to be evaluated.The variables used CANNOT be placeholders like 'x' or 'mike's age', they must be numbers"}}

- search: Search the web for information.
    Takes inputs: {'query': {'description': 'the query', 'type': <class 'str'>}}

The way you use the tools is by specifying a json blob.
Specifically, this json should have a `action` key (name of the tool to use) and a `action

OIDOIOINCOZ
{'max_new_tokens': 3000, 'top_k': None, 'top_p': 0.95, 'typical_p': 0.95, 'temperature': 0.8, 'repetition_penalty': None, 'return_full_text': False, 'truncate': None, 'stop_sequences': ['Observation:'], 'seed': None, 'do_sample': False, 'watermark': False} None


=====Output message of the LLM:=====
 Thought: Now that I know Kyle found 48 shells, I can calculate how many shells Leigh collected by taking one-third of that number.

Action:
{
  "action": "calculator",
  "action_input": {
    "expression": "48 / 3"
  }
}




==Result==
Observation: 16.0
=====Calling LLM with these messages:=====
Solve the following task as best you can. You have access to the following tools:


- calculator: This is a tool that calculates. It can be used to perform simple arithmetic operations.
    Takes inputs: {'expression': {'type': <class 'str'>, 'description': "The expression to be evaluated.The variables used CANNOT be placeholders like 'x' or 'mike's age', they must be numbers"}}

- search: Search the web for information.
    Takes inputs: {'query': {'description': 'the query', 'type': <class 'str'>}}

The way you use the tools is by specifying a json blob.
Specifically, this json should have a `action` key (name of the tool to use) and a `action_input` key 

...

OIDOIOINCOZ
{'max_new_tokens': 3000, 'top_k': None, 'top_p': 0.95, 'typical_p': 0.95, 'temperature': 0.8, 'repetition_penalty': None, 'return_full_text': False, 'truncate': None, 'stop_sequences': ['Observation:'], 'seed': None, 'do_sample': False, 'watermark': False} None


=====Output message of the LLM:=====
 Thought: From the list of winners and their nationalities, I can see that Dmitry Matvienko from Belarus won the competition in 2021. Belarus was a part of the Soviet Union, which is a country that no longer exists. I will now search for the first name of Dmitry Matvienko.

Action:
{
  "action": "calculator",
  "action_input": {
    "expression": "Dmitry Matvienko".split(" ")[1] + " " + Dmitry Matvienko.split(" ")[0]
  }
}


Could not parse the given action: The JSON blob you used is invalid: due to the following error: invalid syntax. Perhaps you forgot a comma? (<unknown>, line 4). Try to correct its formatting..
=====Calling LLM with these messages:=====
Solve the following task as best you can. You have access to the following tools:


- calculator: This is a tool that calculates. It can be used to perform simple arithmetic operations.
    Takes inputs: {'expression': {'type': <class 'str'>, 'description': "The expression to be evaluated.The var

OIDOIOINCOZ
{'max_new_tokens': 3000, 'top_k': None, 'top_p': 0.95, 'typical_p': 0.95, 'temperature': 0.8, 'repetition_penalty': None, 'return_full_text': False, 'truncate': None, 'stop_sequences': ['Observation:'], 'seed': None, 'do_sample': False, 'watermark': False} None


=====Output message of the LLM:=====
 Thought: It seems there was an issue with the previous action input. I will correct it and use the calculator tool to find the first name of Dmitry Matvienko by splitting his full name into first and last names.

Action:
{
  "action": "calculator",
  "action_input": {
    "expression": "Dmitry Matvienko".split(" ")[1] + " " + "Dmitry".split(" ")[0]
  }
}


Could not parse the given action: The JSON blob you used is invalid: due to the following error: malformed node or string on line 4: <ast.BinOp object at 0x2d24b61a0>. Try to correct its formatting..
Failed by reaching max iterations, returning None.
100%|██████████| 90/90 [44:58<00:00, 29.98s/it]


,agent_name,question,prediction,intermediate_steps,parsing_error,iteration_limit_exceeded,agent_error,start_time,end_time,gt_answer,task
0,react_text_mixtral3,Mimi picked up 2 dozen seashells on the beach....,16,[Solve the following task as best you can. You...,False,False,None,2024-03-15 16:37:27,2024-03-15 16:37:35,16.0,gsm8k
1,react_text_mixtral3,Frankie's parents let him have many pets. He h...,"Frankie has 13 pets in total (6 snakes, 7 cats...",[Solve the following task as best you can. You...,False,False,None,2024-03-15 16:37:35,2024-03-15 16:38:11,19.0,gsm8k
2,react_text_mixtral3,"Olaf collects colorful toy cars. At first, his...",None,[Solve the following task as best you can. You...,False,False,None,2024-03-15 16:38:11,2024-03-15 16:38:35,196.0,gsm8k
3,react_text_mixtral3,Emma's bank account has $100 in it. Each day o...,Emma will have $4 remaining in her account.,[Solve the following task as best you can. You...,False,False,None,2024-03-15 16:38:35,2024-03-15 16:38:47,4.0,gsm8k
4,react_text_mixtral3,"Ezekiel hikes as a hobby. This past summer, he...",Ezekiel had to hike 25 kilometers on the third...,[Solve the following task as best you can. You...,False,False,None,2024-03-15 16:38:47,2024-03-15 16:39:00,15.0,gsm8k
...,...,...,...,...,...,...,...,...,...,...,...
85,react_text_mixtral3,How many at bats did the Yankee with the most ...,None,[Solve the following task as best you can. You...,False,False,None,2024-03-15 17:18:14,2024-03-15 17:19:14,519,GAIA
86,react_text_mixtral3,In Audre Lorde’s poem “Father Son and Holy Gho...,"Unfortunately, I can't directly identify the s...",[Solve the following task as best you can. You...,False,False,None,2024-03-15 17:19:14,2024-03-15 17:19:27,2,GAIA
87,react_text_mixtral3,Where were the Vietnamese specimens described ...,None,[Solve the following task as best you can. You...,False,False,None,2024-03-15 17:19:27,2024-03-15 17:20:08,Saint Petersburg,GAIA
88,react_text_mixtral3,What country had the least number of athletes ...,None,[Solve the following task as best you can. You...,False,False,None,2024-03-15 17:20:08,2024-03-15 17:21:11,CUB,GAIA


# Evaluate

In [40]:
import glob

answer_file_path = f"{OUTPUT_DIR}/answers.jsonl"

result_df = pd.concat([pd.read_json(f) for f in glob.glob(f"{OUTPUT_DIR}/*.json")])
result_df = result_df.drop(columns=["start_time", "end_time"])
result_df.to_json(answer_file_path, lines=True, orient="records")

### Exact match for GSM8K

In [ ]:
from scripts.evaluation import score_any_match

results_math = result_df.loc[result_df["task"] == "gsm8k"].copy()
results_math["exact_match"] = -1
results_math["exact_match"] = results_math.apply(
    lambda row: score_any_match(row["prediction"], float(row["gt_answer"])), axis=1
)

Error when extracting string: could not convert string to float: '2U'
Error when extracting string: could not convert string to float: '4x'
Error when extracting string: could not convert string to float: '3r'


In [ ]:
results_math.groupby(["agent_name", "task"])["exact_match"].mean()

agent_name                 task 
code                       gsm8k    0.675
react                      gsm8k    0.375
react_messages_chat_model  gsm8k    0.575
react_messages_openai      gsm8k    0.475
react_prev                 gsm8k    0.425
react_text_chat_model      gsm8k    0.725
react_text_mixtral2        gsm8k    0.375
react_text_normal_model    gsm8k    0.500
react_text_openai          gsm8k    0.400
vanilla_llm                gsm8k    0.650
Name: exact_match, dtype: float64

### LLM as a judge for others

In [ ]:
answers = result_df.loc[result_df["task"] != "gsm8k"].to_dict(orient="records")

In [ ]:
from langchain.chat_models import ChatOpenAI

eval_chat_model = ChatOpenAI(model="gpt-4-1106-preview", temperature=0)
eval_model_name = "GPT4"

In [ ]:
from scripts.evaluation import evaluate_answers
from scripts.prompts import EVALUATION_PROMPT_TEMPLATE

output_file_path = f"{OUTPUT_DIR}/evaluation.jsonl"

run_evaluation = True
if run_evaluation:
    results = await evaluate_answers(
        answers,
        eval_chat_model,
        "GPT4",
        EVALUATION_PROMPT_TEMPLATE,
        output_file_path=output_file_path,
    )

    print("Evaluation is complete!")

Previous evaluations:
                    agent_name  \
0                        react   
1        react_messages_openai   
2    react_messages_chat_model   
3    react_messages_chat_model   
4        react_messages_openai   
..                         ...   
445      react_text_chat_model   
446      react_text_chat_model   
447      react_text_chat_model   
448      react_text_chat_model   
449      react_text_chat_model   

                                              question  \
0    David Pear was a NFL football player, that won...   
1    What American attorney currently represents Ne...   
2    David Pear was a NFL football player, that won...   
3    What American attorney currently represents Ne...   
4    What act of a mythical hunter-warrior of Irish...   
..                                                 ...   
445  Viken Babikian and John Najarian, both work as...   
446  Who produced the eighth and final instalment i...   
447  What American attorney currently represent

  0%|          | 0/50 [00:00<?, ?it/s]

Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example


100%|██████████| 50/50 [00:33<00:00,  1.47it/s]

Evaluation is complete!



Exception in thread Thread-5 (write_line):
Traceback (most recent call last):
  File "/opt/homebrew/Cellar/python@3.11/3.11.6_1/Frameworks/Python.framework/Versions/3.11/lib/python3.11/threading.py", line 1045, in _bootstrap_inner


In [ ]:
results_nonmath = pd.DataFrame.from_dict(results)


def interpret_score(eval_score):
    try:
        return (float(eval_score) - 1) / 4
    except:
        return 0


results_nonmath["eval_score_GPT4"] = results_nonmath["eval_score_GPT4"].apply(
    interpret_score
)
results_nonmath.groupby("agent_name")["eval_score_GPT4"].mean()

agent_name
code                         0.255
react                        0.345
react_messages_chat_model    0.350
react_messages_openai        0.380
react_prev                   0.350
react_text_chat_model        0.470
react_text_mixtral2          0.370
react_text_normal_model      0.375
react_text_openai            0.360
vanilla_llm                  0.420
Name: eval_score_GPT4, dtype: float64

### Aggregate both

In [ ]:
result_df = pd.concat([results_math, results_nonmath])
result_df["aggregate_score"] = (
    result_df["exact_match"].fillna(0) + result_df["eval_score_GPT4"].fillna(0)
) * 100
result_df.groupby(["agent_name", "task"])["aggregate_score"].mean()

agent_name                 task    
code                       GAIA             12.5
                           HotpotQA    34.166667
                           gsm8k            67.5
react                      GAIA             8.75
                           HotpotQA    51.666667
                           gsm8k            37.5
react_messages_chat_model  GAIA             10.0
                           HotpotQA    51.666667
                           gsm8k            57.5
react_messages_openai      GAIA            18.75
                           HotpotQA    50.833333
                           gsm8k            47.5
react_prev                 GAIA             8.75
                           HotpotQA         52.5
                           gsm8k            42.5
react_text_chat_model      GAIA            16.25
                           HotpotQA         67.5
                           gsm8k            72.5
react_text_mixtral2        GAIA             8.75
                           Hotpot

### Display

In [ ]:
import plotly.express as px

aggregate = (
    result_df.groupby(["agent_name", "task"])[["aggregate_score"]].mean().reset_index()
)
aggregate["agent_name"] = aggregate["agent_name"].map(
    {
        "vanilla_llm": "LLM",
        "react": "ReactAgent (Mixtral - messages)",
        "code": "CodeAgent",
        "react_prev": "ReactAgent (Mixtral - text)",
        "react_messages_chat_model": "ReactAgent (Nous - messages)",
        "react_text_chat_model": "ReactAgent (Nous - text)",
        "react_messages_openai": "ReactAgent (OpenAI - messages)",
        "react_text_openai": "ReactAgent (OpenAI - text)",
        "react_text_mixtral2": "ReactAgent (Mixtral2 - text)",
    }
)
fig = px.bar(
    aggregate,
    x="agent_name",
    y="aggregate_score",
    color="task",
    labels={
        "agent_name": "<b>Agent</b>",
        "task": "<b>Task</b>",
        "aggregate_score": "Performance",
        "eval_score_GPT4": "<b>Score</b>",
    },
)
fig.update_layout(
    width=len(aggregate["agent_name"].unique()) * 150 + 200,
    height=600,
    barmode="group",
    bargap=0.35,
    bargroupgap=0.0,
    yaxis_range=[0, 100],
)
fig.update_traces(texttemplate="%{y:.0f}", textposition="outside")
fig.layout.yaxis.ticksuffix = "%"
fig.show()

/Users/aymeric/venvs/ml/lib/python3.11/site-packages/plotly/express/_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.

